In [1]:
import sklearn

In [2]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
from sklearn.model_selection import train_test_split
print('Tensorflow module imported')

NotFoundError: C:\Users\abhi1\AppData\Roaming\Python\Python39\site-packages\tensorflow_text\python\ops\_regex_split_ops.so not found

In [ ]:
tfhub_handle_preprocess = "/Major_Project/bert/bert_preprocess"
# tfhub_handle_encoder = "/Major_Project/bert/small_bert_bert_en_uncased_L-12_H-512_A-8_2"

# tfhub_handle_encoder = "/Major_Project/bert/bert_en_uncased_L-12_H-768_A-12"
tfhub_handle_encoder = "/Major_Project/bert/small_bert_bert_en_uncased_L-8_H-256_A-4_2"

In [ ]:
bert_preprocess_model = hub.KerasLayer(tfhub_handle_preprocess)
bert_model = hub.KerasLayer(tfhub_handle_encoder)
print('encoder and preprocessor ready')

In [ ]:
import pandas as pd
import numpy as np
df = pd.read_csv('/Major_Project/kerala_flood_nepal_earthquake_english_only.csv')

In [ ]:
df
X_train, X_test, y_train, y_test = train_test_split(df['tweet'],df['target'], stratify=df['target'])

In [ ]:
def build_classifier_model():
  text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
  preprocessing_layer = hub.KerasLayer(tfhub_handle_preprocess, name='preprocessing')
  encoder_inputs = preprocessing_layer(text_input)
  encoder = hub.KerasLayer(tfhub_handle_encoder, trainable=True, name='BERT_encoder')
  outputs = encoder(encoder_inputs)
  net = outputs['pooled_output']
  net = tf.keras.layers.Dropout(0.1)(net)
  net = tf.keras.layers.Dense(1, activation=None, name='classifier')(net)
  return tf.keras.Model(text_input, net)
classifier_model = build_classifier_model()

In [ ]:
text_test = ['this is such an amazing movie!']
text_preprocessed = bert_preprocess_model(text_test)
bert_raw_result = classifier_model(tf.constant(text_test))
print(tf.sigmoid(bert_raw_result))

In [ ]:
epochs = 10
# steps_per_epoch = tf.data.experimental.cardinality(X_train).numpy()
# num_train_steps = steps_per_epoch * epochs
# num_warmup_steps = int(0.1*num_train_steps)

init_lr = 3e-5
optimizer = tf.keras.optimizers.Adam(
    learning_rate=0.00001,
    beta_1=0.9,
    beta_2=0.999,
    epsilon=1e-07,
    amsgrad=False,
    name='Adam',
)



In [ ]:
METRICS = [
      tf.keras.metrics.BinaryAccuracy(name='accuracy'),
      tf.keras.metrics.Precision(name='precision'),
      tf.keras.metrics.Recall(name='recall')
]

In [ ]:
classifier_model.compile(optimizer=optimizer,
                         loss = 'binary_crossentropy',
                         metrics= METRICS)
print(f'Training model with {tfhub_handle_encoder}')


In [ ]:
y_train

In [ ]:
history = classifier_model.fit(X_train,
                               y_train,
                               epochs=epochs)

In [ ]:
result = classifier_model.evaluate(X_test, y_test)

In [ ]:
history

In [ ]:
classifier_model.save("/Major_Project/models/bert_success_1/bert_success_1")

In [ ]:
new = tf.keras.models.load_model('/Major_Project/models/bert_success_1/bert_success_1')

In [ ]:
new.summary()

In [ ]:
def print_my_examples(inputs, results):
  result_for_printing = \
    [f'input: {inputs[i]:<30} : score: {results[i][0]:.6f}'
                         for i in range(len(inputs))]
  print(*result_for_printing, sep='\n')
  print()


examples = [
    'My heart goes out to the victims',  # this is the same sentence tried earlier
    'Its a nice day today',
    'The flood is terrible',
    'The movie was okish.',
    'Hello how are you #happybirthday'
]

new.predict([examples[4]])

In [ ]:
new.predict("what a hot summer day")